# Summary

```
def dt(v):
    i = int(v.split(":")[0])
    if i >= 5 and i < 12: return "Morning"
    if i >= 12 and i < 17: return "Afternoon"
    if i >= 17 and i < 20: return "Evening"
    if i >= 20 and i < 5: return "Night"

df['Dep_Time_Applied'] = df['Dep_Time'].apply(dt)
df['Arrival_Time_Applied'] = df['Arrival_Time'].apply(dt)

df[(df.Dep_Time_Applied=='Morning') & (df.Arrival_Time_Applied == 'Evening')].shape[0]
```

# Part1: Obtain MetaData of Practice1.csv

In [1]:
import pandas as pd
# df = pd.read_csv('Practice1.csv')
df = pd.read_csv('https://drive.usercontent.google.com/download?id=1zP38WCdZQ9StsAHTTLk72763QJQkhWrG&export=download&authuser=0')

In [2]:
print(df.columns)
df.shape # (9450, 13)

Index(['Airline', 'Source', 'Destination', 'Route', 'Dep_Time', 'Arrival_Time',
       'Duration', 'Total_Stops', 'Additional_Info', 'Price', 'Month',
       'WeekDay', 'Day'],
      dtype='object')


(9450, 13)

In [3]:
df.head(2)

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Month,WeekDay,Day
0,Jet Airways,Delhi,Banglore,DEL → BOM → COK,20:00,04:25 10 Jun,26h 35m,1 stop,In-flight meal not included,14924,6,Thursday,6.0
1,Jet Airways,Delhi,Cochin,DEL → BOM → COK,16:00,19:00 10 Jun,27h,1 stop,In-flight meal not included,10577,6,Sunday,9.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9450 entries, 0 to 9449
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          9450 non-null   object 
 1   Source           9450 non-null   object 
 2   Destination      9450 non-null   object 
 3   Route            9449 non-null   object 
 4   Dep_Time         9450 non-null   object 
 5   Arrival_Time     9450 non-null   object 
 6   Duration         9450 non-null   object 
 7   Total_Stops      9250 non-null   object 
 8   Additional_Info  9450 non-null   object 
 9   Price            9450 non-null   int64  
 10  Month            9450 non-null   int64  
 11  WeekDay          9450 non-null   object 
 12  Day              9214 non-null   float64
dtypes: float64(1), int64(2), object(10)
memory usage: 959.9+ KB


In [5]:
df.describe() # summary excluded nan values

,Price,Month,Day
count,9450.000000,9450.000000,9214.000000
mean,9027.895556,4.718730,13.517582
std,4466.677471,1.162725,8.459792
min,1759.000000,3.000000,1.000000
25%,5198.000000,3.000000,6.000000
50%,8366.000000,5.000000,12.000000
75%,12373.000000,6.000000,21.000000
max,57209.000000,6.000000,27.000000


# Part2: Direct Questions

##### Q1) What is the average of the flight ticket price? Write your answer correct to two decimal places.

In [6]:
df['Price'].mean() # count, sum, meanMedianStd, minMax

9027.895555555555

##### Q2) During which month did the highest number of flights occur? Months are represented by numerical codes, with January corresponding to 1, February to 2, and so forth.

In [7]:
x = df.Month.value_counts()
x.idxmax(), x.max()

(5, 3092)

In [8]:
df.Month.value_counts()

Month
5    3092
6    3044
3    2388
4     926
Name: count, dtype: int64

##### Q3) Is the average price of flight tickets higher on weekends (Saturday and Sunday) or on weekdays (Remaining 5 days)?

In [9]:
list(df.WeekDay.unique())

['Thursday', 'Sunday', 'Friday', 'Monday', 'Saturday', 'Wednesday', 'Tuesday']

In [10]:
wd = ['Saturday', 'Sunday']
df['day_type'] = df['WeekDay'].apply(lambda x: 'wnd' if x in wd else 'wd')

m = df.groupby('day_type')['Price'].mean()
m

day_type
wd     9015.219666
wnd    9058.016077
Name: Price, dtype: float64

In [11]:
m.wnd.mean() > m.wd.mean()

True

##### Q4) Two of the entries in the 'Additional_Info' column are 'No info' and 'No Info'. Replace all occurrences of 'No Info' with 'No info'. How many flights fall under airline 'IndiGo' and have 'No info' as additional information?

In [12]:
d = {'No Info': 'No info'}

df['Additional_Info_Replaced'] = df['Additional_Info'].replace(d)
df.Additional_Info.unique(), df.Additional_Info_Replaced.unique()

(array(['In-flight meal not included', 'No info',
        'No check-in baggage included', '1 Long layover',
        'Change airports', 'No Info', '2 Long layover', '1 Short layover',
        'Business class'], dtype=object),
 array(['In-flight meal not included', 'No info',
        'No check-in baggage included', '1 Long layover',
        'Change airports', '2 Long layover', '1 Short layover',
        'Business class'], dtype=object))

In [13]:
df[(df.Airline=='IndiGo') & (df.Additional_Info_Replaced=='No info')].shape[0]

1650

##### Q5) Convert the values of 'Duration' into seconds. Enter the average duration (in seconds) of a flight. Enter your answer correct to two decimal places.

In [14]:
# df.Duration.unique() # xh ym
def f(x):
    l = x.split()
    s = 0
    for word in l:
        if word[-1] in ['h', 'H']:
            s += int(word[:-1])*3600
        elif word[-1] in ['m', 'M']:
            s += int(word[:-1])*60
        else:
            s += int(word[:-1])
    return s

df['Duration_In_Seconds'] = df['Duration'].apply(f)
df.Duration_In_Seconds.mean()

38957.93650793651

# Part3: Preprocessing
Apply the following functions to the columns Dep_Time and Arrival_Time:

Transform the values in the 'dep_time' and 'arrival_time' columns to represent the hour component. For instance, if an entry is 10:05 June 13 or 10:05, the corresponding value should be 10. 
Then convert the time into four categories as follows:
- 5 <= hour < 12 = Morning 
- 12 <= hour < 17 = Afternoon 
- 17 <= hour < 20 = Evening 
- 20 <= hour < 5 = Night

Note: Please ensure that you make the changes directly within the dataset and continue to use that modified dataset for subsequent questions.

In [15]:
df.head(2)

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Month,WeekDay,Day,day_type,Additional_Info_Replaced,Duration_In_Seconds
0,Jet Airways,Delhi,Banglore,DEL → BOM → COK,20:00,04:25 10 Jun,26h 35m,1 stop,In-flight meal not included,14924,6,Thursday,6.0,wd,In-flight meal not included,95700
1,Jet Airways,Delhi,Cochin,DEL → BOM → COK,16:00,19:00 10 Jun,27h,1 stop,In-flight meal not included,10577,6,Sunday,9.0,wnd,In-flight meal not included,97200


In [16]:
def dt(v):
    i = int(v.split(":")[0])
    if i >= 5 and i < 12: return "Morning"
    if i >= 12 and i < 17: return "Afternoon"
    if i >= 17 and i < 20: return "Evening"
    if i >= 20 and i < 5: return "Night"

df['Dep_Time_Applied'] = df['Dep_Time'].apply(dt)
df['Arrival_Time_Applied'] = df['Arrival_Time'].apply(dt)

##### Q6) How many flights started in the Morning and arrived the destination at Evening?

In [17]:
df[(df.Dep_Time_Applied=='Morning') & (df.Arrival_Time_Applied == 'Evening')].shape[0]

922

##### Q7) Encode the values of column 'WeekDay' as follows:
- Weekends (Sunday, Saturday) = 1
- all remaining five days = 0
- What is the most frequent (mode) WeekDay?

In [18]:
df.day_type.value_counts()

day_type
wd     6651
wnd    2799
Name: count, dtype: int64

In [19]:
df.day_type.mode()

0    wd
Name: day_type, dtype: object

# Answers
- 1) 9027.895555555555
- 2) May (5th 3092 counts)
- 3) Weekends(Saturday and Sunday) (9058.016077 > 9015.219666)
- 4) 1650
- 5) 38957.93650793651
- 6) 922
- 7) Weekdays (6651 counts)